# ChordAI - Model Training on Google Colab

Bu notebook ChordAI projesinin LSTM modelini Google Colab'da eğitmek için hazırlanmıştır.

## Önemli Not:
**Model architecture TensorFlow.js uyumluluğu için güncellenmiştir!**
- Input layers'a explicit `dtype='int32'` eklendi
- TensorFlow 2.19+ ile uyumlu

## Adımlar:
1. GitHub repo'yu klonla
2. Gerekli paketleri yükle
3. Modeli eğit
4. Model dosyasını indir
5. TensorFlow.js'e dönüştür

**GPU Runtime kullanmayı unutma!**
Runtime > Change runtime type > Hardware accelerator: GPU

## 1. Setup - GitHub Repo Clone

In [ ]:
# Clone GitHub repository
!git clone https://github.com/bufyyy/chordai.git
%cd chordai

## 2. Install Dependencies

In [ ]:
# Install required packages (use Colab's TensorFlow 2.19)
!pip install -q tensorflow numpy pandas matplotlib scikit-learn tensorflowjs

## 3. Check GPU Availability

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

## 4. Verify Dataset

In [ ]:
import json
import os

# Check dataset files
dataset_files = [
    'dataset/train.json',
    'dataset/val.json',
    'dataset/test.json',
    'dataset/chord_vocab.json',
    'dataset/metadata_vocab.json'
]

for file in dataset_files:
    exists = os.path.exists(file)
    size = os.path.getsize(file) if exists else 0
    print(f"{'✓' if exists else '✗'} {file}: {size:,} bytes")

# Load and check sample data
with open('dataset/train.json', 'r') as f:
    train_data = json.load(f)
    print(f"\nTraining samples: {len(train_data)}")
    print("Sample:", train_data[0])

## 5. Train Model

Bu adım 10-20 dakika sürecek (GPU ile).

In [ ]:
%cd model-training

# Train with optimal settings for Colab
!python train_model.py --epochs 50 --batch-size 32 --lr 0.001

## 6. Evaluate Model

In [ ]:
# Evaluate on test set and generate samples
!python evaluate_model.py --model output/chord_model_final.h5 --evaluate --generate --num-samples 10

## 7. View Training Results

In [ ]:
# Load and display training history
with open('output/training_history.json', 'r') as f:
    history = json.load(f)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(history['loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_title('Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(history['accuracy'], label='Train Accuracy')
axes[1].plot(history['val_accuracy'], label='Val Accuracy')
axes[1].set_title('Model Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"\nFinal Training Accuracy: {history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history['val_accuracy'][-1]:.4f}")

## 8. Convert to TensorFlow.js

In [ ]:
# Convert model to TensorFlow.js format with quantization
!python convert_pipeline.py --model output/chord_model_final.h5

## 9. Check Converted Model

In [ ]:
# Check converted files
import os

tfjs_path = '../client/public/model'

print("TensorFlow.js Model Files:")
print("-" * 50)

for root, dirs, files in os.walk(tfjs_path):
    level = root.replace(tfjs_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 2 * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        size = os.path.getsize(file_path)
        size_mb = size / (1024 * 1024)
        print(f"{sub_indent}{file} ({size_mb:.2f} MB)")

# Calculate total size
total_size = 0
for root, dirs, files in os.walk(tfjs_path):
    for file in files:
        total_size += os.path.getsize(os.path.join(root, file))

print(f"\nTotal Model Size: {total_size / (1024 * 1024):.2f} MB")

## 10. Download Files

Model eğitimi tamamlandı! Şimdi dosyaları indir:

1. **Keras Model**: `output/chord_model_final.h5`
2. **TensorFlow.js Model**: `../client/public/model/` klasörünün tamamı
3. **Training History**: `output/training_history.json`
4. **Test Results**: `output/test_results.json`

In [ ]:
# Zip model files for easy download
!zip -r chordai_trained_model.zip output/ ../client/public/model/

print("✅ Model files zipped!")
print("Download: chordai_trained_model.zip")
print("\nYou can also download individual files:")
print("- output/chord_model_final.h5 (Keras model)")
print("- ../client/public/model/ (TensorFlow.js model)")

from google.colab import files
files.download('chordai_trained_model.zip')

## 11. Next Steps

Model başarıyla eğitildi! Şimdi:

1. `chordai_trained_model.zip` dosyasını indir
2. Zip'i aç
3. `client/public/model/` klasörünü projendeki `client/public/model/` ile değiştir
4. GitHub'a push et:
   ```bash
   git add client/public/model/
   git commit -m "Add trained AI model"
   git push
   ```
5. Vercel otomatik deploy edecek
6. Demo mode devre dışı kalacak ve gerçek AI çalışacak! 🎉

---

**Not**: Model dosyaları büyükse Git LFS kullanman gerekebilir.